In [6]:
import pickle
with open('data/labeled_df.pkl', 'rb') as f:
    labeled_df = pickle.load(f)

with open('data/test_xy_for_eval.pkl', 'rb') as f:
    x_test, y_test = pickle.load(f)

with open('data/pseudo_labeled_df.pkl', 'rb') as f:
    unlabeled_df = pickle.load(f)

In [9]:
labeled_df

,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Max,Fwd Packet Length Std,Packet Length Variance,Flow Duration,Flow Bytes/s,Flow Packets/s,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Average Packet Size,Avg Fwd Segment Size,label,source
0,0.496997,0.555556,0.633634,0.000000,0.614615,0.639640,0.792940,0.558019,0.592655,0.455095,0.436982,0.000000,0.829329,0.871872,0.792941,0.871872,0.633634,1,Labeled
1,0.594094,0.484484,0.483483,0.948949,0.479980,0.000000,0.000000,0.604705,0.196073,0.368522,0.378831,0.958959,0.333834,0.492492,0.000000,0.510010,0.483483,1,Labeled
2,0.496997,0.555556,0.633634,0.000000,0.614615,0.639640,0.710711,0.546393,0.603348,0.457934,0.450046,0.000000,0.674675,0.889890,0.710711,0.889890,0.633634,1,Labeled
3,0.885385,0.663664,0.558559,0.000000,0.614615,0.557057,0.908909,0.715454,0.506592,0.184725,0.262280,0.000000,0.941942,0.787788,0.908909,0.788288,0.558559,1,Labeled
4,0.496997,0.555556,0.633634,0.000000,0.614615,0.639640,0.853353,0.524352,0.624415,0.480846,0.474621,0.000000,0.829329,0.905405,0.853353,0.905405,0.633634,1,Labeled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556551,0.885385,0.747748,0.729230,0.000000,0.774775,0.731732,0.938093,0.674810,0.552326,0.381825,0.363198,0.000000,0.976977,0.748248,0.938093,0.748248,0.729230,2,Labeled
556552,0.663664,0.950472,0.945546,0.000000,0.988696,0.990699,0.565568,0.674834,0.531429,0.366979,0.324826,0.000000,0.566066,0.565817,0.565569,0.565635,0.945546,2,Labeled
556553,0.808809,0.725726,0.757622,0.000000,0.745245,0.748498,0.793243,0.674746,0.552285,0.381840,0.351053,0.000000,0.751251,0.736737,0.793244,0.736737,0.757622,2,Labeled
556554,0.885385,0.871371,0.765766,0.000000,0.918919,0.805914,0.938052,0.674744,0.552540,0.381841,0.363222,0.000000,0.976977,0.773783,0.938052,0.773783,0.765766,2,Labeled


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Separate features and labels
x_labeled = labeled_df.drop(columns=["label", "source"])
y_labeled = labeled_df["label"]

x_unlabeled = unlabeled_df.drop(columns=["label", "source"])
y_pseudo = unlabeled_df["label"]  # pseudo-labels

# Train model on true labeled data
model1 = RandomForestClassifier(random_state=42)
model1.fit(x_labeled, y_labeled)

# Predict on unlabeled (pseudo-labeled) data
y_pred_unlabeled = model1.predict(x_unlabeled)

# Compare to pseudo-labels
print("🔍 Step 1: True-label model vs Pseudo-labels")
print(classification_report(y_pseudo, y_pred_unlabeled))


🔍 Step 1: True-label model vs Pseudo-labels
              precision    recall  f1-score   support

         0.0       0.00      1.00      0.00         1
         1.0       0.00      0.00      0.00         6
         2.0       0.39      0.92      0.54     73931
         3.0       0.29      0.02      0.04    151722
         4.0       0.01      0.03      0.02     25396
         5.0       0.00      0.00      0.00     88656

    accuracy                           0.21    339712
   macro avg       0.11      0.33      0.10    339712
weighted avg       0.21      0.21      0.14    339712



In [11]:
# Train on pseudo-labeled data
model2 = RandomForestClassifier(random_state=42)
model2.fit(x_unlabeled, y_pseudo)

# Predict on labeled data
y_pred_labeled = model2.predict(x_test)

# Compare to true labels
print("🔁 Step 2: Pseudo-label model vs True labels")
print(classification_report(y_test, y_pred_labeled))


🔁 Step 2: Pseudo-label model vs True labels
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     12403
           1       0.00      0.00      0.00    115214
           2       0.57      0.96      0.72    226598
           3       0.02      0.37      0.03      1781
           4       0.89      0.49      0.64    142951
           5       0.00      0.00      0.00      1954

    accuracy                           0.58    500901
   macro avg       0.25      0.30      0.23    500901
weighted avg       0.51      0.58      0.51    500901



c:\Users\lucch\Desktop\thesis-datasec\thesis\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\lucch\Desktop\thesis-datasec\thesis\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\lucch\Desktop\thesis-datasec\thesis\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi